# Survival C++ and how to use it to work with data

In order to understand the course material we need to have a grasp on the small subset of C++ that is used. Although C++ is the language of choice for many real-world scientific applications, it is more complex to understand than plain C, and can potentially introduce uncecessary complexity that may get in the way of effectively grasping the course material. This course will use mostly the C subset of the C++ language, however we use C++ compilers and tools so that we can use more advanced C++ features if convenient.

## Comments

Comments in C++ begin with two forward slashes //. Anything after the two forward slashes is ignored for the rest of the line.

```C++
// This is a comment
```

## Statements and code blocks

A statement is a line of code. Since C++ doesn't use new lines to separate statements then every statement must have an ending, C++ uses the semicolon ";" character to end statements.

```C++
// This is a statement that declares and integer with value 2
int a = 2;
```

**Code blocks** are collections of statements enclosed by a starting brace "{", and ending with an end brace "}". Variables declared within code blocks do not exist outside their enclosing code block.

```C++
{ // Starting a code block
    int a = 2;
} // Ending a code block
```

It is good practice to indent the code that lies within a code block, as demonstrated above. The code for functions is also contained within a code block. In fact all the code that runs in a C++ program is launched from a single function called **main**. Here is the source code for a complete C++ program. 

```C++
int main() {
    // Code for the program goes within this code block
    int a = 2;
}
```

There can only be one **main** function in a program.

## Basic data types

There are only really two kinds of data types, integers and floats, with varying numbers of bits to represent them. Every other data type is a reinterpretation or compound mixture of these basic data types.

### Integers

Integers represent whole numbers. There are two main types, signed and unsigned, with varying numbers of bits to represent them. For integers of length **N**, each bit is associated with a power of 2, and the value of the integer is the dot product between the vector of bits (1's and 0's) and a vector of decreasing powers of 2 ranging from $2^{N-1}$ to $2^{0}$ . With signed integers the most common representation is **two's complement**. This is where the first (most significant or highest value) element in the vector of two's is negative.  

<figure style="margin-left:auto; margin-right:auto; width:60%;">
    <img style="vertical-align:middle" src="images/integers.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Signed and unsigned integers with N=8 bits.</figcaption>
</figure>

Using the formulae for values we can derive the largest and smallest values for the integer types.

| | signed | unsigned | 
| :- | :- | :- |
| **smallest value** | $$-2^{N-1}$$ | $$0$$ | 
| **largest value** | $$2^{N-2}-1$$ | $$2^{N-1}$$ | 


There are a number of data types in C that represent integers with varying numbers of bits. Not every integer type means the same number of bits on every platform.

| Nominal number of bits (N) | name of signed form | name of unsigned form | 
| :- | :- | :- |
|8|char|unsigned char| 
|16|short|unsigned short|
|32|int|unsigned int|
|32-64|NA|size_t|
|64|long|unsigned long|

The C statement to create integers is, for example:

```C++
// Creating integers
int a=10;
unsigned int b=10;
```

The character type (char) is just an 8-Bit integer whose value is associated with a lookup table of ASCII characters. We create a character in C using the following form.

```C++
// Create a character
char c = 'a';
```

### IEEE754 Floating point numbers

The IEEE754 standard for floating point numbers was established in 1985 and is the standard used in many applications, including OpenCL implementations. The bits are laid out in three sections: a Sign bit, an Exponent, and a Mantissa. The Sign bit occupies one bit, the Exponent has **NBE** bits and the Mantissa has **NBM** bits. The total number of bits for a floating point number is then NBE+NBM+1. The Exponent **E** is just an unsigned integer created by a dot product of the Exponent bits with a vector of decreasing positive powers of 2 ranging from $2^{\mathrm{NBE}-1}$ to $2^0$. The Mantissa is constructed the same way, but the value is $2^{0}==1$ plus the dot product of the Mantissa bits with a vector of decreasing powers of 2 ranging from $2^{-1}$ to $2^{-\mathrm{NBM}}$. All three components combine together to form the value, as shown below.

<figure style="margin-left:auto; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="images/floating_point.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Floating point representations with differing numbers of bits.</figcaption>
</figure>

The standard reserves special meaning (such as $\pm \infty$, NaN, or subnormal), for when the bits in the Exponent are either all zeros 0 or all 1. Therefore the smallest value is $E=1$ and the largest is $E=(2^{\mathrm{NBE}}-1)-1$. Within these limits we see that a floating point number can describe everything within the following range of numbers:

| | | 
| :- | :- |
| **smallest value** | $$ 2^{1-\mathrm{Bias}}$$ 
| **largest value** | $$ \left (2-2^{\mathrm{-NBM}} \right) 2^{\mathrm{Bias}}$$ 

The **frexp** function in Python and C returns the values $x=0.5 \times (-1)^{S} \times \left ( 1.0 + \sum^{\mathrm{NBM}-1}_{i=0} B_i 2^{i-\mathrm{NBM}} \right )$ and $y=E-\mathrm{Bias}+1.$ to make $$\mathrm{Value} = x \times 2^{y}$$

In [30]:
import math
(x,y) = math.frexp(1.0)
print(x,y)

0.5 1


For any given floating point number $f$ within the limits, the next represented floating point number is exactly at a spacing of $\Delta f=2^{(E-\mathrm{Bias}-\mathrm{NBM})}=2^{(y-1-\mathrm{NBM})}$. A twofold change in spacing occurs at every power of two.

#### Example

For 32-bit floating point numbers the number of bits in the Exponent is $\mathrm{NBE}=8$ and the number of bits in the Mantissa is $\mathrm{NBM=23}$, therefore $\mathrm{Bias}=127$. The smallest (normal) floating point representation is $2^{1-\mathrm{Bias}}=1.1754944\times 10^{-38}$ and the largest (normal) floating point representation is $3.4028235\times10^{38}$. For values around $f=1.0$, where $E-\mathrm{Bias}=0$ the spacing to the next floating point representation is $\Delta f=2^{-23} \approx 1.1920929 \times 10^{-7}$.

### Floating point numbers in C

The C floating point data types for varying numbers of bits are as follows:

| Nominal number of bits (NBM+NBE+1) | name |  
| :- | :- | 
|16|half||
|32|float| 
|64|double|
|64-128| long double|

Declaring a floating point number in C may be done as follows:

```C++

// half precision (16-bit), only on some platforms
half a=2.0;

// single precision (32-bit)
float b=2.0;

// double precision (64-bit)
double c=2.0;

// quadruple precision (64-bit)
long double d=2.0;

```


## Pointers and the stack

Thus far we have been initialising integers and floats as statements in a program. These values require memory somewhere in which to store the bits. When the program is executed, the operating system allocates memory for the values in a pre-prepared and reserved area of memory for the program called the **stack**. In C/C++ one can get the starting address (as an integer) of the allocated memory, and this address is called a **pointer**.

Pointers can store the address of any memory allocation. The pointer type determines how the memory will be accessed, i.e as a float, as an integer etc. We create a pointer to a particular data type by declaring a variable as usual and putting a **\*** in front of the variable name to indicate that it is a pointer to a variable of that type.

```C++
int *p;  // Create a pointer to an integer called p
int a=2; // Create an integer and give it a value of 2
```

The address operator **&** gets the address of allocated memory, which can be assigned to the pointer.

```C++
p = &a;  // Get the address of the integer and assign it to p
```

Finally, the de-referencing operator **\*** can access the value pointed to by a pointer.

```C++
int y = *p; // Access the value pointed to by p and assign to y
```

There is a danger here, if **p** was not assigned to an address and we try to de-reference it, then the result will lead to a memory error and a potential crash of the program. There is a pointer type **void\*** that can point to any memory allocation.

```C++
void *p = &a;
```

Pointers may be set to **NULL** when they are not pointing to anything useful.

```C++
void *p = NULL;
```

## Functions

A function is some code that does a task and can be called any number of times. Functions take zero or more arguments or variables as input and return zero or one output. Here are some example functions:

```C++
void simple() { // Takes no arguments and returns no values, we use void as the return type
    int a=2;
}

int get_int() { // Takes zero arguments and returns an integer, notice we have to declare the return type
    return 0; // The return is necessary
}
```

We call these functions from the program as follows;

```C++
// Call a function that doesn't return anything
simple();

// Call a function that returns an integer
int y = get_int();
```

The limitation on one return value isn't so bad because we can pass pointers into the function and use the dereferencing operator to set values. Notice that we have to set the data types on the inputs and outputs.

```C++
float more_complex(float *p, float a) { // Takes two arguments, a pointer and a float
    *p = a; // dereference p and set the contents to a
    return *p + 1;
}
```

Such a function would be used as follows:

```C++
float a = 1.0;
float b = 2.0;

// We send the address of b and the value of a to the function.
// The function will then fill b with a copy of the contents of a and return a+1
float c = more_complex(&b, a);
```

## Printing values

The function [**printf**](https://www.cplusplus.com/reference/cstdio/printf) offers a way to print variables to the command line. You need to include **\<cstdio\>** in the header of the program, like this:
    
```C++
#include <cstdio>
#include <iostream>
```

The function is to be used like follows:

```C++
char a='s';
float x=5.0;
float y=6.0;
printf("%c", a); // Print a to the screen with its memory interpreted as an integer
printf("%f %f\n", x, y); // Print x and y to the screen with its memory interpreted as floats
printf("%i %i\n", y, x); // Print y and x to the screen with their memory interpreted as integers 
```

The first argument is the format string and it can be followed by any number of arguments. Notice the newline character **'\n'** in the format string to make a new line at the end of the printing. The percent characters **\%** are placeholders for variables in the arguments and describe how the arguments should be interpreted and displayed.
    
In C++ there is also the **cout** functionality to send output to the standard output stream. Just ensure you have included **\<iostream\>** in the header region.
    
```C++
// Print x and a space, then y and a new line
std::cout << x << " " < y << std::endl;
```

## Math and other operators in C++

Math operators in C and C++ have the following form in descending order of priority. A more comprehensive list can be found [here](https://en.cppreference.com/w/c/language/operator_precedence).

| Operator | Explanation |  
| :- | :- | 
| ++x | increment x by 1 and return the incremented x |
| x++ | return x and then increment x by 1 |
| --x | decrement x by 1 and return decremented x |
| x-- | return x and then decrement x by 1 |
| x * y | multiply x and y |
| x / y | divide x by y |
| x % y | return the remainder when x is divided by y |
| x > y | return 1 if x is greater than y, zero otherwise |
| x < y | return 1 if x is less than y, zero otherwise |
| x >= y | return 1 if x is greater than or equal to y, zero otherwise |
| x <= y | return 1 if x is less than or equal y, zero otherwise |
| x == y | return 1 if x is equal in value to y, zero otherwise |
| x != y | return 1 if x is not equal in value to y, zero otherwise |
| x \|\| y | return 1 if either x OR y are not zero, return zero otherwise |
| x && y | return 1 if both x AND y are not zero, return zero otherwise |

```C++
// Example use of math operators
int x=2;
int y=3;

printf("%d\n", x>y); // will print 0, why?
```

## Flow control with **If** statements and **for** loops

**If** statements and **for** loops are implemented using code blocks. **If** statements may optionally have one or more **"else if"** statements, and 0 or 1 **"else"** catch all statements. Wether or not the code block is entered depends on the result of the expression immediately after the **if**, a value of 1 means True and 0 means False. For example:

```C++
// Demonstrating if statements
int x=2;
int y=3;

// Using an if statement
if (x > y) {
    // This line won't be executed because x is smaller than y
    printf("%i is greater than %i", x, y);
}

// Using an if statement with an else
if (x > y) {
    printf("%i is greater than %i", x, y);
} else {
    // This statement gets executed
    printf("%i is less than or equal to %i", x, y);  
}

// Using an if statement with an else if and an else
if (x > y) {
    printf("%i is greater than %i", x, y);
} else if (x==y) {
    printf("%i is equal to %i", x, y);  
} else {
    // This statement gets executed
    printf("%i is less than %i", x, y);  
}
```

When writing **if** statements one has to be very careful to adequately handle all of the different permuations that the choices provide. **For** loops are constructed in the following matter

```C++
// Demonstrating a for loop
int N=12;

//   starting condition; continue condition; increment
for (int x=0; x<N; x++) {
    // Execute this code block each time we go around the loop
    printf("x is now %i", x);
}
```

For loops usually have a loop variable. The starting condition is $x=0$ and we are free to create the loop variable at that time. Next is the continuing condition, here it is (x<N) which means that the loop will continue while $(x<N)$ evaluates to 1 (True). The increment condition is what happens to the loop variable at the end of each iteration.

Of course **for** loops may be nested, like this:

```C++
// Demonstrating a 2D for loop

// initialise loop limits
int M=10, N=12;

//   starting condition; continue condition; increment
for (int m=0; m<M; m++) {
    for (int n=0; n<N; n++) {
        // Execute this code block each time we go around the loop
        printf("m is %i, n is %i\n", x);
    }
}
```

The printf statement in the innermost loop will execute 120 times.

## Arrays and memory allocations

Sometimes we want to allocate memory for more than one element. If we know the size of the array beforehand and it is small (i.e. less than the stack size limit for programs in the OS) then we can allocate an array on the stack as follows:

```C++
int a[10]; // Creates an array of 10 elements
```

When we do this the variable **a** is actually a pointer that points to the first address of a memory allocation of 10 integers. We can use the indexing operator **[]** with the pointer to get at first and subsequent elements of the allocation. 

```C++
// Print the first element in the array at index 0
printf("%f", a[0]); 
```

Indicies in C++ start at 0 and array bounds are not checked, so if we go beyond the 10th element and try to access memory that is not allocated for the array, then the OS might detect the access violation and crash the program.

### Dynamically allocating arrays with **calloc** and **free**, **new** and **delete**

Sometimes we don't know how big an array is until runtime, or we know that the array is too big to fit into the stack. In such cases we can allocate memory at runtime from a large pool of available memory called the **heap**. Two functions to reserve memory are **[calloc](https://www.cplusplus.com/reference/cstdlib/calloc/)** and **[malloc](https://www.cplusplus.com/reference/cstdlib/malloc/)**. I prefer calloc because unlike malloc it always initialises the memory contents with zeros. Memory allocation with calloc looks like the following:

```C++
size_t N = 1024;
void *p = calloc(N, sizeof(float)); // Allocates memory for 1024 floats
```

This allocates memory for 1024 floats. The sizeof function determines the size of argument in Bytes. The pointer returned by calloc is a void pointer, but we can use the casting operator **()** to convert the void pointer to a pointer that we need. In this case it is **float\***.

```C++
size_t N = 1024;
float *p = (float*)calloc(N, sizeof(float)); // Allocate memory and cast to float* in one step.
```

Now we can use the indexing operator **[]** to access elements in the allocated memory relative to the position of the first address, which is **p**.

```C++
// Print the first element
printf("%f", p[0]); 
```

Memory that is allocated like this must be manually deallocated. The **free** function takes a pointer to allocated memory and frees it for use again. Failure to free allocated memory when we no longer need it can result in what is known as a memory leak.

```C++
free(p);
```

In C++ there is also the pair **new** and **delete**. These operate in much the same way as malloc, where initialisation of the allocation to zero is not assured.

```C++
// Allocate memory
float* p = new float[N];

// Print the first element
printf("%f", p[0]); 

// Deallocate memory
delete[] p;
```

For all the examples above we could have used any other data type such as **double** or **int** instead of **float**.

### Working with multi-dimensional arrays (tensors)

Now we have a way of allocating and accessing memory in a one-dimensional array. How do we apply this to higher-dimensional structures like matrices and tensors? Fortunately we don't need multi-dimensional indexing machinery, all we need to do is think of the memory allocation as if it were "folded" into a higher-dimensional structure. Shown below is a memory allocation of 24 elements. Below it are two 3D arrays of size **N** = (3,4,2), constructed by regarding the memory allocation as if it were "folded" according **row-major** and **column-major** ordering.

<figure style="margin-left:auto; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="images/col_major_row_major_indexing.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Row-major and column-major representations of a multi-dimensional array.</figcaption>
</figure>

If you look carefully with the folded representations you can see that a step by 1 along any dimension corresponds to a **stride** of 1 or more elements in the 1D array. For example a step of 1 along dimension 1 requires a **stride of 2** elements in row-major ordering and **3** elements in column-major ordering. The stride for each dimension is encapsulated in the strides vector **S**. In this example **S** = (8,2,1) for row-major and **S** = (1,3,12) for column-major representations. For any (zero-based) coordinate vector **C** the position **P** in the underlying memory allocation is just the dot product between the coordinate vector and the strides vector.

$$P=C \cdot S$$

For example, take the coordinates **C=(1,3,1)**. Under row major ordering with **S=(8,2,1)**, the position is

$$P_{\mathrm{row-major}} = (1 \times 8) + (3 \times 2) + (1 \times 1) = 15.$$

Under column-major ordering with **S=(1,3,12)** the position is now

$$P_{\mathrm{col-major}} = (1 \times 1) + (3 \times 3) + (1 \times 12) = 22.$$

This way of thinking about multi-dimensional arrays is really powerful, because in order to step in any direction in the multi-dimensional representation all we need to do is know what the stride is for that dimension. 

#### Constructing strides vectors

Constructing the strides vector in each ordering scenario is straightforward. For **row-major ordering** we fill the stride vector from right to left and place a stride of 1 in the last dimension. In this case $S_{2}=1$. Then we obtain the previous element $S_{1}$ using the pattern.

$$S_{i-1} = S_{i} N_{i}.$$

Repeat the
pattern until the stride vector is filled. 

Filling the strides vector is much the same for **column-major ordering**, but we fill from left to right. Starting with $S_{0}=1$ the next element is filled using the pattern

$$S_{i+1} = S_{i} N_{i}.$$

## Strings

C++ has a rich set of commands to handle strings. In the C subset a string is just an array of characters with a null character **'\0'** at the end to signify the end of the string. The null character is required by many string handling functions otherwise it can run off the end of the array. We can declare a string as follows:

```C++
// Declare a C string
char f[] = "first string"; // String whose contents may be modified
const char p[] = "second string"; // String whose contents may not be modified
const char *q = "third string"; // String whose contents may not be modified

// The values f and p are just pointers to the first address in the array of characters

// Print the string, this prints all characters up to the null character 
printf("%s\n", p);

// Print element 0 as a character 
printf("%c\n", p[0]);
```

When strings are declared in this way there is no need to explicitly assign the null character to the last element as the compiler does it for you. However inserting a null character may become necessary if you have an array of characters and turn that array into a string. 

## Reading and writing data to files

There are much better ways to write binary data to files using self-descriptive formats like HDF5. However sometimes you might be stuck and just need a quick way to get your data in an out of your program. Within the C subset of the C++ language there is a simple way to read and write binary data in the format that your architecture supports.

### Opening and closing files

The **fopen** function and **fclose** pair of functions open and close files. The **fopen** function takes the form

```C++
FILE *fopen(const char *fname, const char *mode)
```

where the string **fname** is the file name and **mode** determines how the file is opened. A pointer to a **FILE** object is returned by **fopen**. There are two main modes to access a file, text mode and binary mode. Linux and Windows systems differ in how to represent a new line, on Windows a new line means '\r\n', whereas on Linux it means '\n'. Opening a file in text mode means lines are read and written in an OS-specific way. Opening a file in binary mode means that everything is read or written **as is** from the file, and no special interpretation is applied to new line characters. Here are the main modes useful for working with files:

| Mode | Explanation |  
| :- | :- | 
| "w" | open a file for writing in text mode, overwrite if file exists |
| "wb" | open a file for writing in binary mode, overwrite if file exists |
| "r" | open a file for reading in text mode |
| "rb" | open a file for reading in binary mode |
| "a" | open a file for appending in text mode, create if file does not exist |
| "ab" | open a file for appending in binary mode, create if file does not exist |

Adding a plus sign **"+"** to the mode allows updates to happen to the file as well. It is kind of like a read/write modifier.

| Mode | Explanation |  
| :- | :- | 
| "w+" | open a file for writing and updating in text mode, overwrite if file exists |
| "w+b" | open a file for writing and updating in binary mode, overwrite if file exists |
| "r+" | open a file for reading and updating in text mode |
| "r+b" | open a file for reading and updating in binary mode |
| "a+" | open a file for appending and updating in text mode, create if file does not exist |
| "a+b" | open a file for appending and updating in binary mode, create if file does not exist |

For example with this code we open a file for writing, you will need to include **\<cstdio\>** in the header of the program though.

```C++
// Open a file for writing in binary mode
FILE *fp = fopen("myfile.dat", "wb");
```

Of course, if this process fails somehow then **fp** will be filled with a **NULL** pointer. So you can check to see if something went wrong. 

When we are finished with files they must be closed with **fclose**.

```C++
// Open a file for writing in binary mode
fclose(fp);
```

### Positioning within a file using **fseek** and **ftell**

Every open file has a "current position" associated with it. This hearkens back to the days when files were read and written to tape. We can "rewind" or "fast-forward" this position using the **fseek** function and report on the current position using **ftell**. The signature for both functions is as follows:

```C++
int fseek(FILE *fp, long int relative_offset, int position);
long int ftell(FILE *fp);
```

The value **relative_offset** is the offset to seek to relative to **position**, and position is one of **SEEK_SET** for the beginning of the file; **SEEK_CUR** for the current position; and **SEEK_END** for the end location. The function **ftell** just tells the absolute location of the current position in the file.

For example, in the code below we seek to the start and end locations in the file and report on their location.

```C++
// Open the file
FILE *fp = fopen("file.dat", "rb");

// Seek to the end of the file, 
// this position is beyond all information in the file
fseek(fp, 0, SEEK_END);

// Get how many bytes long the file is
long int nbytes = ftell(fp);

// Seek to the beginning of the file
fseek(fp, 0, SEEK_SET);

// Close the file
fclose(fp);
```


### Reading and writing binary data with **fread** and **fwrite**

Once a file is open in binary mode you can use the **fread** and **fwrite** functions to read and write data. Both functions have the same signature.

```C++
// Binary read
size_t fread(void *p, size_t element_size, size_t nelements, FILE *fp);
// Binary write
size_t fwrite(void *p, size_t element_size, size_t nelements, FILE *fp);
```

In both instances **p** must be a pointer to allocated memory with size (element_size\*nelements), and the file must point to a valid open file.

### Complete example

Here is a complete example of writing 5 floats to a file and reading it back in to another array.

```C++
// Define the size of the array
int N=5;
const char *fname = "filename.dat"

// Fill the array
float* src = (float*)calloc(N, sizeof(float));
for (int n=0; n<N; n++) {
    src[n] = (float)n;
}

// Open the file and write the array to it
FILE *fp = fopen(fname, "wb");
fwrite(src, sizeof(float), (size_t)N, fp);

// free the source array and close the file
fclose(fp);
free(src);

// Open the file for reading
fp = fopen(fname, "rb");

// Get the number of bytes in the file
fseek(fp, 0, SEEK_END);
long int nbytes = ftell(fp);

// Number of elements in an array
long int nelements = nbytes/sizeof(float);

// Allocate and read from the file
float *dst = (float*)calloc(nelements, sizeof(float))
fread(dst, sizeof(float), nelements, fp);

// Free the destination array and close the file
free(dst);
fclose(fp);
```

* Debugging with GDB
* Exercise: debugging with GDB